
## A Sketch-based Index for Correlated Dataset Search

### Motivation

With the ever-rising amount of data available, recent researched explored queries where we want to enlarge our dataset by finding related data. This means finding the top-k tables which are joinable and correlated to our inital dataset. As these searches can be quite long they propose a more efficient and effective way of finding these tables than just naively querying over the whole data collection, which might take quite a bit. Their proposed idea has shown to achieve better results than other approaches regarding ranking accruacy and recall. 
In this notebook we want to offer you an understandable explanation of the proposed Solution and hope you are able to grasp all of it.

### Naive approach
Before going into detail about the papers algorithm, we want to first show the way the searches are carried out naively.
In order to do that we check all tables from the database we want to search any categorical column for matches to our query table. The amount of Entries found is the Joinability. When we then calculate the correlation of our query tables numerical values and their counterparts in the searched database. For every table we then have a Joinability and a correlation which we want to weight according to our preferences. <br>
Allthough leading to perfect results this procedure will be to time and space consuming in the long term. The following algortithm is beware of the size of modern day databases and therefore aims to achieve a higher speed at the cost of as little accuracy as possible.


### Discovering Data with joinable keys and correlated data.

Steps of the algorithm:
1. Build an inverted index of all tables in DB
    - Transform larger tables into 2-col-tables: One categorical (string) column and one numerical column
    - Take n rows of each 2-col-table, this is our sketch
    - Calculate for each row in the sketch the deviation of the numerical value from the mean of the entire sketch
    - Use the categorical key combined with the deviation as the term for the inverted index
    
    
2. Query
    - Create the sketch of the query table
    - Search in the inverted index for the table with the most overlap of keys combined with corresponding deviation of mean
    

### Building the index (details):

1. All tables with more then 2 columns are transformed into multiple tables with two columns:
    - All categorical columns are combined with all columns containig numerical values (crossproduct)
<br>

2. Pick a specific sample (=sketch) of size n per table
    - To pick the rows from the table the values in the categorical column are hashed
    - The n smallest hashes are contained in the sketch
    - This way the samples are comparable 
<br>

3. All categorical keys (k) are modified according to their numerical values (c_k)
    - First the mean of the numerical values of the sketch has to be calculated
    - If the numerical value of a row is below or above the mean, k is categorized in k- or k+ respectively
    - This is used to identify correlation
    - The term for k-/k+ keys in the index is k + "-1"/ k + "+1" (e.g. "Germany" with c_k above mean -> "Germany+1")



In [ ]:
import os
import pandas as pd
from collections import Counter
from collections import defaultdict
from pathlib import Path
from qcr import load_index, get_kc, get_c, hash_function, build_index, key_labeling, create_sketch, cross_product_tables, print_dict

#### Step 1
In this example we want to find data correlating to the life expectancy in certain countries. To be able to search the other tables we start by building the index. Therefore, we first load 3 tables containing example data.

In [ ]:
input_table = pd.read_csv('data/test_table.csv')    # import table
input_table.columns.name = 'testTable'             # assign name
table_id = input_table.columns.name                 # store name for later
display(input_table) 

In order to process tables containing multiple columns, we have to split them up into sub tables consisting of a categorical column (our key) and a numerical column (feature).

In [ ]:
cat_col = get_kc(input_table)
num_col = get_c(input_table)

print_dict(cat_col, "cat_col")
print_dict(num_col, "num_col")

tables = cross_product_tables(cat_col, num_col, table_id)

table1 = tables[0] # for out example, we only build the sketch of the first table by hand
table1

As you can see, the table now consists of the only categorical column of the original table, and one of the numerical columns. Also, the table name is now testTable_Country_Alcohol. The schema for this is `<originalTableName>_<categoricalColName>_<numericColName>`. This way we can later conclude which columns of which table are correlated to our query.


#### Step 2
Now we have tables containing only one categorical and one numerical column. <br>
We can start hashing the categorical column. For performance reasons it is advised to limit the sketch size. <br>
Although not necessary in this small case, we want to emphasize the scalability of this approach and use the limit of 5.

In [ ]:
# we work on a copy of the table, to keep the variables clean for later use
hashed_table1 = table1.copy()

# create hashes
hashed_table1['hashed_keys'] = table1['Country'].map(hash_function)
print(hashed_table1)

# apply sketch size
sketch = hashed_table1.nsmallest(5, 'hashed_keys')
print('\nsketch:')
sketch

The sketch of table 1 now consists of the 5 countries whose hashed keys are lowest.


#### Step 3
Now the paper labels the hashed keys according to the mean of the values.<br>
To increase Readability we use the actual keys instead of the hashed keys here.<br>
Any row with its value below our mean has its key concatenated with "-1", while any row with its value above the mean has its key concatenated with "+1".

In [ ]:
# first we calculate the mean of all values of this tables numeric column
mean = sketch['Alcohol'].mean()

# again we work on a copy of the sketch, to keep the variables clean
labeled_sketch = sketch.copy()

# then label key by > median (key+1) or < median (key-1)
labeled_sketch['labeled_keys'] = [f'{key}{"+1" if value > mean else "-1"}' for key, value, hash_key in sketch.values]

print(labeled_sketch)

As we can see Nicaragua and Armenia have been labeled "-1" and Equatorial Guinea "+1".<br>
(If the labeled_keys seem to be in a second table: this is not the case. The table is simply split into two lines, if it is too wide for the screen.)

#### Step 4
We finally can complete the construction of our inverted index by merging our sketches into our full inverted index. 

In [ ]:
table_id = labeled_sketch.columns.name

# initialize new inverted index
inverted_index = defaultdict(set)

# now each labed key (=term) is inserted into the dict with the table name it came from
for term in labeled_sketch['labeled_keys']:
    inverted_index[term].add(table_id)

print_dict(inverted_index, "inverted index")

As we can see the keys chosen from the table by the sketch have been tagged and are saved with a reference to the table they originate from. <br>
We now want to build the sketch for our whole database. Our Code provides the build_index function which performs the above shown procedure for a list of tables and merges them into one index.<br>
It also stores the index as a pickle file on disc. In case there is an old file stored, we remove that connection.



In [ ]:
# delete old inverted index in case it exists
if os.path.exists("index.pickle"):
    Path("index.pickle").unlink()

# build index of all sub-tables of the original input-table
build_index([input_table] ,n=5)
inverted_index = load_index()
inverted_index = dict(sorted(inverted_index.items()))  # sort index for better comparison

print_dict(inverted_index, 'inverted Index')

This is now our inverted index for the given tables using the sketch size of 5. We can see that Armenia has an above average value in 'testTable_Country_BMI', so its inhabitants have a BMI above the average of the sketched countries. On the other hand its size and Alcohol consumption is below average. <br>

So, having completed the construction of our index for the database, we can search over the inverted index using our query table. 


### Querying the index
As we also had tables with countries as a categorical value, we must use them here too. So we decide to search for values correlated to our initial values, having the same categorical value.

#### Step 1

We start by building the inverted index for the sketch of our query data.

In [ ]:
# load query table: (key & target)
query = pd.read_csv('data/q.csv')
query.columns.name = 'q'
display(query)



As we can see this table has a categorical and a numercial column. We therefore can use it to search our inverted index. Here we want to find tables containing value correlated to our target column (Life expectancy).

In [ ]:
# as above:
# 1. build sketch of query table
sketch = create_sketch(query['Country'], query['Life expectancy '], hash_function, n=5)
# 2. generate terms
search_terms = key_labeling(sketch)
print("search terms:")
search_terms

#### Negative correlation
As we also might be interested in negative correlation, we negate the values and build a second set of labeled keys.
This results in two query tables: one for positive correlation, one for negative.

In [ ]:

inverse_search_terms = key_labeling(
    list(map((lambda key_value: (key_value[0], -key_value[1])), sketch))
    ) # same function as above, input key is inverted
print("inverse search terms:")
inverse_search_terms

#### Step 2
Using our new search terms, we can now query the index. We then count which table comes up how often in the results.
we limit the output to the ten most correlated tables

In [ ]:

inverted_index = load_index()

result = Counter()
result.update(
    "+:" + table_id for term in search_terms for table_id in inverted_index[term]
)
result.update(
    "-:" + table_id for term in inverse_search_terms for table_id in inverted_index[term]
)

sketch = result.most_common(10)
sketch

### So, what do these results show us? <br>
Our first entry is of the country-bmi column combination. The "+" indicates a positive correlation, concatenated to the tableID. The 5 at the end shows that 5 entries of the sketch of this table match the query tables entries and are marked the same according to the sketches averages. This tells us, that the values are correlated (when one rises, the other does too). Therefore, the table is likely to enrich our data.<br>
For the country-alcohol column combination are 4 negative correlated entries and 1 positive correlated. Although not as clearly correlated as the first table, it might still be useful to join them to enrich our data.<br>
For the country-area column combination, there are 3 positively and 2 negatively correlated entries. This is almost a 50/50 distribution, therefore the values might just a well be random. It would be unwise to add these columns as they might not help us predict our target value. <br>
We have now decided to join our query table with the BMI table, as it's highly correlated and also join with the Alcohol column which still has a high enough chance to probably provide informational gain.

In [ ]:
result = query.join(input_table[['Country','BMI']].set_index('Country'), on='Country' , how='left', lsuffix='_who', rsuffix='_kaggle')

result = result.join(input_table[['Country','Alcohol']].set_index('Country'), on='Country' , how='left', lsuffix='_who', rsuffix='_kaggle')

result

#### Use Case

One use case for correllated Dataset search might be Machine Learning. One could enrich the Training data with New data found by querying the target column of the ML-Objective. For example, one might want to train a machine learning model to predict the life expectancy given attributes like child mortality, wealth,etc. Then one could search for columns correlated to life expectancy and would likely find data like BMI and Alcohol. The BMI and Alcohol columns would the be added to the feature data. There is a downside to this method, which is an implicit data leak from the target data to features, because the new data is correlated to the target. Therefore the user needs to be careful when performing experiments and comparing results. In the most extreme case, one could find a table of life expectancy by the query if it exists in the dataset, since the target has the highest correlation to itself. A query result with perfect correlation should therefore be discarded. However, there is no perfect solution possible. Our team is currently in the process of preparing experiments using these methods and precautions.

### Conclusion

In this notebook we showed how the approach proposed in the paper works and outlined its advantages towards naive implementations. It is noteable that the procedure takes correlation and inability into account.
Joinability of categorical columns is secured, by using these categorical columns for keys in the inverted index. Marking the distribution of the values and comparing them with the search columns numeric values ensures that the output tables & columns correlate.

#### Scalability
We already implemented a version of the code presented in SQL. With that Version we were able to build the complete inverted index for the Gittables dataset of 1 million tables in about 4 hours (sketch size = 128). However, due to necessatity of data cleaning, we could not perform any experiments yet or use the dresden datatset. But this demonstrates that our code is efficient and scales well to large datasets.

When querying with values of the entire life expectancy table (see data/long_query.csv) searching the entire inverted index and thus the entire database of 1 million Gittables takes about 10 seconds and yields results with high joinability and correlation.


In [ ]:
# You can play around with the results here

long_query = pd.read_csv('data/long_query.csv', sep=';') # Query used
result = pd.read_csv('data/long_query_result.csv', sep=';') # Top 100 result ids
top_result = pd.read_csv('data/growth_rate_tables_licensed_HDI_2015_1.csv', sep=';') # Top 1 result table
display(long_query)
display(result)
display(top_result)
joined = long_query.join(top_result[["0", "5"]].set_index("0"), on='Country', how='left', lsuffix='_query', rsuffix='_result')
display(joined)